# Imports

In [ ]:
from src.convolution.numerical import self_convolution, convolution
from src.spectrum import (
    LinealEnergySpectrum,
    SpectrumData,
    SpectrumValueType,
    SpecificEnergySpectrum,
    specific_energy_spectum,
    lineal_energy_spectum,
)
from src.paths import project_dir
from src.probability import cfds_with_progress

In [ ]:
import numpy as np
import matplotlib

%matplotlib inline
import matplotlib.pylab as plt
from tqdm.notebook import tqdm

In [ ]:
!python -V

# Single event spectra for Cs137 (d=1um)

## f1 y spectrum

In [ ]:
spectrum_y_f1_raw = LinealEnergySpectrum.from_csv(
    project_dir / "data" / "interim" / "Cs137_ydy.csv",
    delimiter="\t",
    value_type=SpectrumValueType.ydy,
)
spectrum_y_f1_raw.norm

In [ ]:
spectrum_y_f1_raw.data.binning_type

In [ ]:
spectrum_y_f1_raw.data.bin_centers[1:] / spectrum_y_f1_raw.data.bin_centers[:-1]

In [ ]:
new_bin_ratio = np.average(
    spectrum_y_f1_raw.data.bin_centers[1:] / spectrum_y_f1_raw.data.bin_centers[:-1]
)
new_bin_ratio

In [ ]:
np.log(spectrum_y_f1_raw.data.bin_centers)

In [ ]:
np.log(spectrum_y_f1_raw.data.bin_centers).sum()

In [ ]:
np.log(new_bin_ratio)

In [ ]:
log_a0 = (
    np.log(spectrum_y_f1_raw.data.bin_centers).mean()
    - np.log(new_bin_ratio) * (spectrum_y_f1_raw.data.num_bins + 1) / 2
)
log_a0

In [ ]:
new_log_bin_centers = np.linspace(
    start=log_a0 + np.log(new_bin_ratio),
    stop=log_a0 + np.log(new_bin_ratio) * (spectrum_y_f1_raw.data.num_bins),
    num=spectrum_y_f1_raw.data.num_bins,
    endpoint=True,
)
new_log_bin_centers

In [ ]:
spectrum_y_f1 = LinealEnergySpectrum(
    data=SpectrumData(
        bin_centers=np.exp(new_log_bin_centers),
        bin_values_freq=spectrum_y_f1_raw.data.bin_values_freq,
    )
)
spectrum_y_f1.norm, spectrum_y_f1.data.binning_type

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
ax[0].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.fy,
    label="freq",
)
ax[0].set_ylabel("fy")
ax[1].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.yfy,
    label="yfy",
)
ax[1].axvline(
    spectrum_y_f1.yF,
    color="k",
    linestyle="--",
    label=f"yF {spectrum_y_f1.yF:2.2f} keV/um",
)
ax[1].set_ylabel("yfy")
ax[2].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.ydy,
    label="ydy",
)
ax[2].axvline(
    spectrum_y_f1.yD,
    color="b",
    linestyle="--",
    label=f"yD {spectrum_y_f1.yD:2.2f} keV/um",
)
ax[2].set_ylabel("ydy")
for a in ax:
    a.set_xscale("log")
    a.grid()
    a.set_xlabel("y [keV/um]")
    a.legend()

## f1 z spectrum

In [ ]:
site_diam_um = 1.0
spectrum_z_f1 = specific_energy_spectum(spectrum_y_f1, site_diam_um=site_diam_um)
spectrum_z_f1.data.binning_type

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
ax[0].step(
    spectrum_z_f1.data.bin_edges[1:],
    spectrum_z_f1.fz,
    label="freq",
)
ax[0].set_ylabel("fz")
ax[1].step(
    spectrum_z_f1.data.bin_edges[1:],
    spectrum_z_f1.zfz,
    label="zfz",
)
ax[1].axvline(
    spectrum_z_f1.zF,
    color="k",
    linestyle="--",
    label=f"zF {spectrum_z_f1.zF:2.2f} Gy",
)
ax[1].set_ylabel("zfz")
ax[2].step(
    spectrum_z_f1.data.bin_edges[1:],
    spectrum_z_f1.zdz,
    label="zdz",
)
ax[2].axvline(
    spectrum_z_f1.zD,
    color="b",
    linestyle="--",
    label=f"zD {spectrum_z_f1.zD:2.2f} Gy",
)
ax[2].set_ylabel("zdz")
for a in ax:
    a.set_xscale("log")
    a.grid()
    a.set_xlabel("z [Gy]")
    a.legend()

# Convolution

In [ ]:
new_log_bin_centers_for_conv = np.linspace(
    start=log_a0 + np.log(new_bin_ratio),
    stop=log_a0 + np.log(new_bin_ratio) * (spectrum_y_f1_raw.data.num_bins + 4),
    num=spectrum_y_f1_raw.data.num_bins + 4,
    endpoint=True,
)
new_log_bin_centers_for_conv
y_for_conv = np.exp(new_log_bin_centers_for_conv)
y_for_conv

## convolution f2 = f1*f1 for y

In [ ]:
# we apply convolution on f(y), not on yfy or ydy
spectrum_y_f1_function = lambda x: spectrum_y_f1.data.bin_value(
    x, spectrum_value_type=SpectrumValueType.fy
)

expected_f2_domain = (
    spectrum_y_f1.data.bin_edges[0] * 2,
    spectrum_y_f1.data.bin_edges[-1] * 2.0,
)

convolution_integration_limits = (
    spectrum_y_f1.data.bin_edges[0] / 8,
    spectrum_y_f1.data.bin_edges[-1] * 8,
)
integral_kwargs = {
    "limit": 800,
    "points": np.geomspace(*convolution_integration_limits, 400),
}
spectrum_y_f2_values_and_errors = [
    self_convolution(
        spectrum_y_f1_function,
        lower_limit=convolution_integration_limits[0],
        upper_limit=convolution_integration_limits[1],
        kwargs=integral_kwargs,
    )(y)
    for y in tqdm(y_for_conv)
]

In [ ]:
# by comparing the integration error with f2 values we check the numerical integration accuracy
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))
ax.plot(
    y_for_conv,
    [item[0] for item in spectrum_y_f2_values_and_errors],
    ".",
    label="f2 values",
)
ax.plot(
    y_for_conv,
    [item[1] for item in spectrum_y_f2_values_and_errors],
    ".",
    label="f2 error (from integration)",
)
ax.grid()
ax.set_xlabel("y [keV/um]")
ax.legend()
ax.set_xscale("log")
ax.set_yscale("log")

## f2 y spectrum

In [ ]:
spectrum_y_f2 = LinealEnergySpectrum(
    data=SpectrumData(
        bin_centers=y_for_conv,
        bin_values_freq=np.array([item[0] for item in spectrum_y_f2_values_and_errors]),
    )
)

In [ ]:
spectrum_y_f2.yF, spectrum_y_f2.yD

In [ ]:
# check if yF is doubled
spectrum_y_f2.yF / spectrum_y_f1.yF

In [ ]:
# check norm
spectrum_y_f2.norm

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
ax[0].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.fy,
    label="f1",
)
ax[0].step(
    spectrum_y_f2.data.bin_edges[1:],
    spectrum_y_f2.fy,
    label="f2",
)
ax[0].set_ylabel("fy")
ax[1].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.yfy,
    label=f"f1, yF = {spectrum_y_f1.yF:2.2f} keV/um",
)
ax[1].step(
    spectrum_y_f2.data.bin_edges[1:],
    spectrum_y_f2.yfy,
    label=f"f2, yF = {spectrum_y_f2.yF:2.2f} keV/um",
)
ax[1].set_ylabel("yfy")
ax[2].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.ydy,
    label=f"f1 yD = {spectrum_y_f1.yD:2.2f} keV/um",
)
ax[2].step(
    spectrum_y_f2.data.bin_edges[1:],
    spectrum_y_f2.ydy,
    label=f"f2 yD = {spectrum_y_f2.yD:2.2f} keV/um",
)
ax[2].set_ylabel("ydy")
for a in ax:
    a.set_xscale("log")
    a.grid()
    a.set_xlabel("y [keV/um]")
    a.legend()

## convolution f3 = f2*f1 for y

In [ ]:
# we apply convolution on f(y), not on yfy or ydy
spectrum_y_f2_function = lambda x: spectrum_y_f2.data.bin_value(
    x, spectrum_value_type=SpectrumValueType.fy
)

expected_f3_domain = (
    spectrum_y_f1.data.bin_edges[0] * 2,
    spectrum_y_f2.data.bin_edges[-1] * 2,
)

convolution_integration_limits = (
    spectrum_y_f1.data.bin_edges[0] / 8,
    spectrum_y_f2.data.bin_edges[-1] * 8,
)
integral_kwargs = {
    "limit": 800,
    "points": np.geomspace(*convolution_integration_limits, 400),
}
spectrum_y_f3_values_and_errors = [
    convolution(
        spectrum_y_f1_function,
        spectrum_y_f2_function,
        lower_limit=convolution_integration_limits[0],
        upper_limit=convolution_integration_limits[1],
        kwargs=integral_kwargs,
    )(y)
    for y in tqdm(y_for_conv)
]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))
ax.plot(
    y_for_conv,
    [item[0] for item in spectrum_y_f3_values_and_errors],
    ".",
    label="f3 values",
)
ax.plot(
    y_for_conv,
    [item[1] for item in spectrum_y_f3_values_and_errors],
    ".",
    label="f3 error (from integration)",
)
ax.grid()
ax.set_xlabel("y [keV/um]")
ax.legend()
ax.set_xscale("log")
ax.set_yscale("log")

## f3 y spectrum

In [ ]:
spectrum_y_f3 = LinealEnergySpectrum(
    data=SpectrumData(
        bin_centers=y_for_conv,
        bin_values_freq=np.array([item[0] for item in spectrum_y_f3_values_and_errors]),
    )
)
spectrum_y_f3.norm, spectrum_y_f3.yF, spectrum_y_f3.yD

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
ax[0].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.fy,
    label="f1",
)
ax[0].step(
    spectrum_y_f2.data.bin_edges[1:],
    spectrum_y_f2.fy,
    label="f2",
)
ax[0].step(
    spectrum_y_f3.data.bin_edges[1:],
    spectrum_y_f3.fy,
    label="f3",
)
ax[0].set_ylabel("fy")
ax[1].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.yfy,
    label=f"f1, yF = {spectrum_y_f1.yF:2.2f} keV/um",
)
ax[1].step(
    spectrum_y_f2.data.bin_edges[1:],
    spectrum_y_f2.yfy,
    label=f"f2, yF = {spectrum_y_f2.yF:2.2f} keV/um",
)
ax[1].step(
    spectrum_y_f3.data.bin_edges[1:],
    spectrum_y_f3.yfy,
    label=f"f3, yF = {spectrum_y_f3.yF:2.2f} keV/um",
)
ax[1].set_ylabel("yfy")
ax[2].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.ydy,
    label=f"f1 yD = {spectrum_y_f1.yD:2.2f} keV/um",
)
ax[2].step(
    spectrum_y_f2.data.bin_edges[1:],
    spectrum_y_f2.ydy,
    label=f"f2 yD = {spectrum_y_f2.yD:2.2f} keV/um",
)
ax[2].step(
    spectrum_y_f3.data.bin_edges[1:],
    spectrum_y_f3.ydy,
    label=f"f3 yD = {spectrum_y_f3.yD:2.2f} keV/um",
)
ax[2].set_ylabel("ydy")
for a in ax:
    a.set_xscale("log")
    a.grid()
    a.set_xlabel("y [keV/um]")
    a.legend()

## convolution f4 = f2*f2 for y

In [ ]:
expected_f4_domain = (
    spectrum_y_f2.data.bin_edges[0] * 2,
    spectrum_y_f2.data.bin_edges[-1] * 2.0,
)

convolution_integration_limits = (
    spectrum_y_f2.data.bin_edges[0] / 8,
    spectrum_y_f2.data.bin_edges[-1] * 8,
)
integral_kwargs = {
    "limit": 800,
    "points": np.geomspace(*convolution_integration_limits, 400),
}
spectrum_y_f4_values_and_errors = [
    self_convolution(
        spectrum_y_f2_function,
        lower_limit=convolution_integration_limits[0],
        upper_limit=convolution_integration_limits[1],
        kwargs=integral_kwargs,
    )(y)
    for y in tqdm(y_for_conv)
]

In [ ]:
# by comparing the integration error with f2 values we check the numerical integration accuracy
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))
ax.plot(
    y_for_conv,
    [item[0] for item in spectrum_y_f4_values_and_errors],
    ".",
    label="f4 values",
)
ax.plot(
    y_for_conv,
    [item[1] for item in spectrum_y_f4_values_and_errors],
    ".",
    label="f4 error (from integration)",
)
ax.grid()
ax.set_xlabel("y [keV/um]")
ax.legend()
ax.set_xscale("log")
ax.set_yscale("log")

## f4 y spectrum

In [ ]:
spectrum_y_f4 = LinealEnergySpectrum(
    data=SpectrumData(
        bin_centers=y_for_conv,
        bin_values_freq=np.array([item[0] for item in spectrum_y_f4_values_and_errors]),
    )
)
spectrum_y_f4.norm, spectrum_y_f4.yF, spectrum_y_f4.yD

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
ax[0].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.fy,
    label="f1",
)
ax[0].step(
    spectrum_y_f2.data.bin_edges[1:],
    spectrum_y_f2.fy,
    label="f2",
)
ax[0].step(
    spectrum_y_f3.data.bin_edges[1:],
    spectrum_y_f3.fy,
    label="f3",
)
ax[0].step(
    spectrum_y_f4.data.bin_edges[1:],
    spectrum_y_f4.fy,
    label="f4",
)
ax[0].set_ylabel("fy")
ax[1].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.yfy,
    label=f"f1, yF = {spectrum_y_f1.yF:2.2f} keV/um",
)
ax[1].step(
    spectrum_y_f2.data.bin_edges[1:],
    spectrum_y_f2.yfy,
    label=f"f2, yF = {spectrum_y_f2.yF:2.2f} keV/um",
)
ax[1].step(
    spectrum_y_f3.data.bin_edges[1:],
    spectrum_y_f3.yfy,
    label=f"f3, yF = {spectrum_y_f3.yF:2.2f} keV/um",
)
ax[1].step(
    spectrum_y_f4.data.bin_edges[1:],
    spectrum_y_f4.yfy,
    label=f"f4, yF = {spectrum_y_f4.yF:2.2f} keV/um",
)
ax[1].set_ylabel("yfy")
ax[2].step(
    spectrum_y_f1.data.bin_edges[1:],
    spectrum_y_f1.ydy,
    label=f"f1 yD = {spectrum_y_f1.yD:2.2f} keV/um",
)
ax[2].step(
    spectrum_y_f2.data.bin_edges[1:],
    spectrum_y_f2.ydy,
    label=f"f2 yD = {spectrum_y_f2.yD:2.2f} keV/um",
)
ax[2].step(
    spectrum_y_f3.data.bin_edges[1:],
    spectrum_y_f3.ydy,
    label=f"f3 yD = {spectrum_y_f3.yD:2.2f} keV/um",
)
ax[2].step(
    spectrum_y_f4.data.bin_edges[1:],
    spectrum_y_f4.ydy,
    label=f"f4 yD = {spectrum_y_f4.yD:2.2f} keV/um",
)
ax[2].set_ylabel("ydy")
for a in ax:
    a.set_xscale("log")
    a.grid()
    a.set_xlabel("y [keV/um]")
    a.legend()

## f2, f3, f4 z spectrum

In [ ]:
spectrum_z_f2 = SpecificEnergySpectrum(
    data=SpectrumData(
        bin_centers=0.204 * spectrum_y_f2.y / site_diam_um**2,
        bin_values_freq=spectrum_y_f2.fy,
    )
)
spectrum_z_f3 = SpecificEnergySpectrum(
    data=SpectrumData(
        bin_centers=0.204 * spectrum_y_f3.y / site_diam_um**2,
        bin_values_freq=spectrum_y_f3.fy,
    )
)
spectrum_z_f4 = SpecificEnergySpectrum(
    data=SpectrumData(
        bin_centers=0.204 * spectrum_y_f4.y / site_diam_um**2,
        bin_values_freq=spectrum_y_f4.fy,
    )
)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
ax[0].step(
    spectrum_z_f1.data.bin_edges[1:],
    spectrum_z_f1.fz,
    label="f1",
)
ax[0].step(
    spectrum_z_f2.data.bin_edges[1:],
    spectrum_z_f2.fz,
    label="f2",
)
ax[0].step(
    spectrum_z_f3.data.bin_edges[1:],
    spectrum_z_f3.fz,
    label="f3",
)
ax[0].step(
    spectrum_z_f4.data.bin_edges[1:],
    spectrum_z_f4.fz,
    label="f4",
)
ax[0].set_ylabel("fz")
ax[1].step(
    spectrum_z_f1.data.bin_edges[1:],
    spectrum_z_f1.zfz,
    label=f"f1, zF = {spectrum_z_f1.zF:2.2f} Gy",
)
ax[1].step(
    spectrum_z_f2.data.bin_edges[1:],
    spectrum_z_f2.zfz,
    label=f"f2, zF = {spectrum_z_f2.zF:2.2f} Gy",
)
ax[1].step(
    spectrum_z_f3.data.bin_edges[1:],
    spectrum_z_f3.zfz,
    label=f"f3, zF = {spectrum_z_f3.zF:2.2f} Gy",
)
ax[1].step(
    spectrum_z_f4.data.bin_edges[1:],
    spectrum_z_f4.zfz,
    label=f"f4, zF = {spectrum_z_f4.zF:2.2f} Gy",
)
ax[1].set_ylabel("zfz")
ax[2].step(
    spectrum_z_f1.data.bin_edges[1:],
    spectrum_z_f1.zdz,
    label=f"f1 zD = {spectrum_z_f1.zD:2.2f} Gy",
)
ax[2].step(
    spectrum_z_f2.data.bin_edges[1:],
    spectrum_z_f2.zdz,
    label=f"f2 zD = {spectrum_z_f2.zD:2.2f} Gy",
)
ax[2].step(
    spectrum_z_f3.data.bin_edges[1:],
    spectrum_z_f3.zdz,
    label=f"f3 zD = {spectrum_z_f3.zD:2.2f} Gy",
)
ax[2].step(
    spectrum_z_f4.data.bin_edges[1:],
    spectrum_z_f4.zdz,
    label=f"f4 zD = {spectrum_z_f4.zD:2.2f} Gy",
)
ax[2].set_ylabel("zdz")
for a in ax:
    a.set_xscale("log")
    a.grid()
    a.set_xlabel("z [Gy]")
    a.legend()

# Dose dependent  z spectra

In [ ]:
D_Gy = 0.05  # Gy (50 mGy)
D_Gy

In [ ]:
# mean number of events in the site
n = D_Gy / spectrum_z_f1.zF
n

In [ ]:
# In microdosimetry the low dose is the dose at which sensitive site is affected just once.
# However, since the cells are hit independently, even at low doses target can be hit two or
# more times. Therefore, the statistical criterion for low dose is that 90% of the affected targets
# are hit just once. This happens when D ≤ 0.2 zF .

In [ ]:
D_Gy, 0.2 * spectrum_z_f1.zF

In [ ]:
# trying to evaluate n f1(z) + n^2/2 f2(z), we are missing here f0(z)
n, n**2 / 2, n**3 / 6, n**4 / 24

In [ ]:
# z_Gy = np.geomspace(start=1e-2, stop=10, num=300)
# z = 0.204 * y / diam**2
z_Gy = 0.204 * y_for_conv / spectrum_z_f1.site_diam_um**2
spectrum_z_fn_values = (
    n * spectrum_z_f1.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.fz)
    + n**2
    / 2
    * spectrum_z_f2.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.fz)
    + n**3
    / 6
    * spectrum_z_f3.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.fz)
    + n**4
    / 24
    * spectrum_z_f4.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.fz)
)
spectrum_z_fn_values *= np.exp(-n)

In [ ]:
spectrum_z_fn_not_normalized = SpecificEnergySpectrum(
    data=SpectrumData(bin_centers=z_Gy, bin_values_freq=spectrum_z_fn_values),
    site_diam_um=spectrum_z_f1.site_diam_um,
)

In [ ]:
spectrum_z_fn = SpecificEnergySpectrum(
    data=SpectrumData(
        bin_centers=spectrum_z_fn_not_normalized.data.bin_centers,
        bin_values_freq=spectrum_z_fn_not_normalized.data.bin_values_freq,
    ),
    site_diam_um=spectrum_z_fn_not_normalized.site_diam_um,
)

In [ ]:
spectrum_z_fn.zF, spectrum_z_fn.zD

In [ ]:
spectrum_z_fn.norm

In [ ]:
z_Gy = np.geomspace(start=1e-2, stop=10, num=300)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
ax[0].plot(
    z_Gy,
    spectrum_z_f1.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.fz)
    * n
    * np.exp(-n),
    label="f1  * n * np.exp(-n)",
)
ax[0].plot(
    z_Gy,
    spectrum_z_f2.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.fz)
    * (n**2 / 2)
    * np.exp(-n),
    label="f2 * (n**2 / 2) * np.exp(-n)",
)
ax[0].plot(
    z_Gy,
    spectrum_z_f3.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.fz)
    * (n**3 / 6)
    * np.exp(-n),
    label="f3 * (n**3 / 6) * np.exp(-n)",
)
ax[0].plot(
    z_Gy,
    spectrum_z_f4.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.fz)
    * (n**4 / 24)
    * np.exp(-n),
    label="f4 * (n**4 / 24) * np.exp(-n)",
)
ax[0].plot(
    z_Gy,
    spectrum_z_fn.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.fz),
    label="fn",
)
ax[0].set_ylabel("fz")
ax[1].plot(
    z_Gy,
    spectrum_z_f1.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.zfz)
    * n
    * np.exp(-n),
    label="f1 * n * np.exp(-n)",
)
ax[1].plot(
    z_Gy,
    spectrum_z_f2.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.zfz)
    * (n**2 / 2)
    * np.exp(-n),
    label="f2 * (n**2 / 2) * np.exp(-n)",
)
ax[1].plot(
    z_Gy,
    spectrum_z_f3.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.zfz)
    * (n**3 / 6)
    * np.exp(-n),
    label="f3 * (n**3 / 6) * np.exp(-n)",
)
ax[1].plot(
    z_Gy,
    spectrum_z_f4.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.zfz)
    * (n**4 / 24)
    * np.exp(-n),
    label="f4 * (n**4 / 24) * np.exp(-n)",
)
ax[1].plot(
    z_Gy,
    spectrum_z_fn.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.zfz),
    label="fn",
)
ax[1].set_ylabel("zfz")
ax[2].plot(
    z_Gy,
    spectrum_z_f1.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.zdz)
    * n
    * np.exp(-n),
    label="f1 * n * np.exp(-n)",
)
ax[2].plot(
    z_Gy,
    spectrum_z_f2.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.zdz)
    * (n**2 / 2)
    * np.exp(-n),
    label="f2 * (n**2 / 2) * np.exp(-n)",
)
ax[2].plot(
    z_Gy,
    spectrum_z_f3.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.zdz)
    * (n**3 / 6)
    * np.exp(-n),
    label="f3 * (n**3 / 6) * np.exp(-n)",
)
ax[2].plot(
    z_Gy,
    spectrum_z_f4.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.zdz)
    * (n**4 / 24)
    * np.exp(-n),
    label="f4 * (n**4 / 24) * np.exp(-n)",
)
ax[2].plot(
    z_Gy,
    spectrum_z_fn.data.bin_values(z_Gy, spectrum_value_type=SpectrumValueType.zdz),
    label="fn",
)
ax[2].set_ylabel("zdz")
for a in ax:
    a.set_xscale("log")
    a.grid()
    a.set_xlabel("z [Gy]")
    a.legend()

In [ ]:
spectrum_z_fn.norm

# Dose dependent y spectra

In [ ]:
spectrum_y_fn_notnormalized = lineal_energy_spectum(spectrum_z_fn)
spectrum_y_fn_notnormalized.norm

In [ ]:
spectrum_y_fn = LinealEnergySpectrum(
    data=SpectrumData(
        bin_centers=spectrum_y_fn_notnormalized.data.bin_centers,
        bin_values_freq=spectrum_y_fn_notnormalized.data.bin_values_freq
        / spectrum_y_fn_notnormalized.norm,
    )
)
spectrum_y_fn.norm

In [ ]:
y_keV_um = np.geomspace(start=0.05, stop=50, num=600)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
ax[0].plot(
    y_keV_um,
    spectrum_y_f1.data.bin_values(y_keV_um, spectrum_value_type=SpectrumValueType.fy),
    label="f1",
)
ax[0].plot(
    y_keV_um,
    spectrum_y_f2.data.bin_values(y_keV_um, spectrum_value_type=SpectrumValueType.fy),
    label="f2",
)
ax[0].plot(
    y_keV_um,
    spectrum_y_fn.data.bin_values(y_keV_um, spectrum_value_type=SpectrumValueType.fy),
    label="fn",
)
ax[0].set_ylabel("fy")
ax[1].plot(
    y_keV_um,
    spectrum_y_f1.data.bin_values(y_keV_um, spectrum_value_type=SpectrumValueType.yfy),
    label="f1",
)
ax[1].plot(
    y_keV_um,
    spectrum_y_f2.data.bin_values(y_keV_um, spectrum_value_type=SpectrumValueType.yfy),
    label="f2",
)
ax[1].plot(
    y_keV_um,
    spectrum_y_fn.data.bin_values(y_keV_um, spectrum_value_type=SpectrumValueType.yfy),
    label="fn",
)
ax[1].set_ylabel("yfy")
ax[2].plot(
    y_keV_um,
    spectrum_y_f1.data.bin_values(y_keV_um, spectrum_value_type=SpectrumValueType.ydy),
    label="f1",
)
ax[2].plot(
    y_keV_um,
    spectrum_y_f2.data.bin_values(y_keV_um, spectrum_value_type=SpectrumValueType.ydy),
    label="f2",
)
ax[2].plot(
    y_keV_um,
    spectrum_y_fn.data.bin_values(y_keV_um, spectrum_value_type=SpectrumValueType.ydy),
    label="fn",
)
ax[2].set_ylabel("ydy")
for a in ax:
    a.set_xscale("log")
    a.grid()
    a.set_xlabel("y [keV/um]")
    a.legend()

In [ ]:
spectrum_y_f1.yF, spectrum_y_f2.yF, spectrum_y_fn.yF

In [ ]:
y_keV_um = np.geomspace(start=1e-1, stop=1e1, num=100)
integral_kwargs = {"limit": 2000}
cfds_y_fn_with_errors = cfds_with_progress(
    y_keV_um, spectrum_y_fn.data, include_error=True, kwargs=integral_kwargs
)

In [ ]:
cfds_y_f1_with_errors = cfds_with_progress(
    y_keV_um, spectrum_y_f1.data, include_error=True, kwargs=integral_kwargs
)

In [ ]:
fix, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))
ax.errorbar(
    y_keV_um,
    cfds_y_fn_with_errors[:, 0],
    yerr=cfds_y_fn_with_errors[:, 1],
    fmt=".",
    label="fn",
)
ax.errorbar(
    y_keV_um,
    cfds_y_f1_with_errors[:, 0],
    yerr=cfds_y_f1_with_errors[:, 1],
    fmt=".",
    label="f1",
)
ax.grid()
ax.legend()
# ax.set_xscale("log")
ax.set_xlabel("y [keV/um]")
ax.set_ylabel("CFD(y)")
# ax.set_ylim(0.9, None);
# ax.set_xlim(0, 8);